In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path
from platform import system
from difflib import get_close_matches

import util
import lines
import group
import label
import records
import extract
import date

In [2]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir, suffix=".pdf")
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[0]

indexes_path = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/tesseract_data_frames/"
output_dir = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/extracted_indexes/"

if system()=="Windows":
    indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
    output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

indexes_csv = util.list_files(indexes_path, suffix=".csv")

display(test_dfs, file)

['.././test_files/tesseract_data_frames/\\LS_index_1920.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986_split.csv']

'.././test_files/tesseract_data_frames/\\LS_index_1920.csv'

In [3]:
pdf_file = test_files[1]
display(test_files, pdf_file)

['.././test_files/\\Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/\\LS_index_1920.pdf',
 '.././test_files/\\LS_index_1920_split.pdf',
 '.././test_files/\\LS_index_1921.pdf',
 '.././test_files/\\LS_index_1937.pdf',
 '.././test_files/\\LS_index_1944.pdf',
 '.././test_files/\\LS_index_1944_split.pdf',
 '.././test_files/\\LS_index_1976.pdf',
 '.././test_files/\\LS_index_1986.pdf',
 '.././test_files/\\LS_index_1986_split.pdf']

'.././test_files/\\LS_index_1920.pdf'

In [4]:
# fitz mode
start_page = 3

pdf_words, pdf_dicts = util.read_pdf(pdf_file, start_page)

w_df = lines.make_words_df(pdf_words, start_page)
l_d = lines.make_lines_df_from_dicts(pdf_dicts, start_page)
l = lines.merge_close_lines(l_d)
l = lines.remove_useless_lines(l)

r_df = extract.extract_indexes(w_df, l, pdf_file, "fitz")
r_df

Reading pdf from .././test_files/\LS_index_1920.pdf
...
Finished reading 9 page(s)


,country,region,text,date,year,page,extracted_date,full_text
0,AUsTRIA,,Order of the Federal Department for Social Adm...,10.11.,1920,3,10th November 1920,Order of the Federal Department for Social Adm...
1,BELGIUM,,Royal Order instituting an Industrial Medical ...,5.6.,1919,3,5th June 1919,Royal Order instituting an Industrial Medical ...
2,BELGIUM,,Ministerial Order in pursuance of Section 81 o...,28.4.,1884,3,28th April 1884,Ministerial Order in pursuance of Section 81 o...
3,BELGIUM,,Royal Order defining the to be taken by the in...,5.9.,1919,3,5th September 1919,Royal Order defining the to be taken by the in...
4,BELGIUM,,Royal Decree respecting electrical plant in co...,,,3,,Royal Decree respecting electrical plant in co...
...,...,...,...,...,...,...,...,...
203,URUGIJAY,,Decree issuing new regulations under the Act r...,21.5.,1920,11,21st May 1920,Decree issuing new regulations under the Act r...
204,URUGIJAY,,Decree respecting the supervision of hours of ...,18.6.,1920,11,i8th June 1920,Decree respecting the supervision of hours of ...
205,URUGIJAY,,Decree to include messengers under the Eight-H...,18.10.,1920,11,i8th October 1920,Decree to include messengers under the Eight-H...
206,URUGIJAY,,"Act respecting occupational accidents. T., Ur.r",15.11.,1920,11,15th November 1920,. Act respecting occupational accidents. 15th ...


In [6]:
extract.extract_indexes_pdf(pdf_file, start_page)

Reading pdf from .././test_files/\LS_index_1920.pdf
...
Finished reading 9 page(s)


,country,region,text,date,year,page,extracted_date,full_text
0,AUsTRIA,,Order of the Federal Department for Social Adm...,10.11.,1920,3,10th November 1920,Order of the Federal Department for Social Adm...
1,BELGIUM,,Royal Order instituting an Industrial Medical ...,5.6.,1919,3,5th June 1919,Royal Order instituting an Industrial Medical ...
2,BELGIUM,,Ministerial Order in pursuance of Section 81 o...,28.4.,1884,3,28th April 1884,Ministerial Order in pursuance of Section 81 o...
3,BELGIUM,,Royal Order defining the to be taken by the in...,5.9.,1919,3,5th September 1919,Royal Order defining the to be taken by the in...
4,BELGIUM,,Royal Decree respecting electrical plant in co...,,,3,,Royal Decree respecting electrical plant in co...
...,...,...,...,...,...,...,...,...
203,URUGIJAY,,Decree issuing new regulations under the Act r...,21.5.,1920,11,21st May 1920,Decree issuing new regulations under the Act r...
204,URUGIJAY,,Decree respecting the supervision of hours of ...,18.6.,1920,11,i8th June 1920,Decree respecting the supervision of hours of ...
205,URUGIJAY,,Decree to include messengers under the Eight-H...,18.10.,1920,11,i8th October 1920,Decree to include messengers under the Eight-H...
206,URUGIJAY,,"Act respecting occupational accidents. T., Ur.r",15.11.,1920,11,15th November 1920,. Act respecting occupational accidents. 15th ...


In [6]:
b_x0, b_x1, c0_c = group.group_line_starts_ends(l, "fitz")
bor = lines.make_borders_df(b_x0, b_x1)

In [7]:
# tess mode
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df, "tess")
borders = lines.make_borders_df(bins_x0, bins_x1)

rec_df = extract.extract_indexes(pdf_df, lines_df, file, "tess")
rec_df

,country,region,text,date,year,page,extracted_date,full_text
0,INTERNATIONAL,,Convention between France and Poland respectin...,3.9.,1919,1,3fd September 1919,Convention between France and Poland respectin...
1,INTERNATIONAL,,"Labour Treaty between France and Italy. T., In...",30.9.,1910,1,30th September I9IQ,Labour Treaty between France and Italy. 30th S...
2,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,27.11.,1919,1,27th November 1919,Convention between the Minister of Foreign Aff...
3,INTERNATIONAL,,Convention between France and the Czecho- Slov...,20.3.,1920,1,zoth March 1920,Convention between France and the Czecho- Slov...
4,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,26.3.,1920,1,26th March 1920,Convention between the Minister of Foreign Aff...
...,...,...,...,...,...,...,...,...
238,URUGUAY,,Decree issuing new regulations under the Actr ...,21.5.,1920,11,21st May 1920,Decree issuing new regulations under the Actr ...
239,URUGUAY,,Decree respecting the supervision of hours of ...,18.6.,1920,11,18th June 1920,Decree respecting the supervision of hours of ...
240,URUGUAY,,Decree to include messengers under the Eight- ...,18.10.,1920,11,18th October 1920,Decree to include messengers under the Eight- ...
241,URUGUAY,,Act respecting occupational accidents. “r5th ‘...,15.11.,1920,11,r5th November 1920,Act respecting occupational accidents. “r5th ‘...


In [7]:
extract.extract_indexes_tess(file)

,country,region,text,date,year,page,extracted_date,full_text
0,INTERNATIONAL,,Convention between France and Poland respectin...,3.9.,1919,1,3fd September 1919,Convention between France and Poland respectin...
1,INTERNATIONAL,,"Labour Treaty between France and Italy. T., In...",30.9.,1910,1,30th September I9IQ,Labour Treaty between France and Italy. 30th S...
2,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,27.11.,1919,1,27th November 1919,Convention between the Minister of Foreign Aff...
3,INTERNATIONAL,,Convention between France and the Czecho- Slov...,20.3.,1920,1,zoth March 1920,Convention between France and the Czecho- Slov...
4,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,26.3.,1920,1,26th March 1920,Convention between the Minister of Foreign Aff...
...,...,...,...,...,...,...,...,...
238,URUGUAY,,Decree issuing new regulations under the Actr ...,21.5.,1920,11,21st May 1920,Decree issuing new regulations under the Actr ...
239,URUGUAY,,Decree respecting the supervision of hours of ...,18.6.,1920,11,18th June 1920,Decree respecting the supervision of hours of ...
240,URUGUAY,,Decree to include messengers under the Eight- ...,18.10.,1920,11,18th October 1920,Decree to include messengers under the Eight- ...
241,URUGUAY,,Act respecting occupational accidents. “r5th ‘...,15.11.,1920,11,r5th November 1920,Act respecting occupational accidents. “r5th ‘...


## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung FIXED
- Date-Type-Erkennung fehlerhaft
    - 1920 FIXED
- 1920 Seite 11 Uruguay
    - unvollständige Datums-Erkennung FIXED
    

## Todo
- doppelseitige Extraktion funktioniert auch im fitz-Modus
- Kommandozeilenprogramm